In [43]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [44]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [46]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=20):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=20):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [47]:
#read in chicago validation labels
chicago_labels = gpd.read_file('data/round-2/chicago-validation-labels-round2-230608.geojson')
chicago_labels = chicago_labels.drop(columns=['time_created'])
chicago_labels.head()

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,2548,1259,Milton,CurbRamp,41.893814,-88.103554,JqRFV1QowOOjFb-TIMnzMw,None,1.0,f,None,debris,POINT (-88.10355 41.89381)
1,Devon Snyder,2549,1259,Milton,SurfaceProblem,41.893818,-88.103470,7aric67UimoidY9pcCuWiQ,None,1.0,f,grass,None,POINT (-88.10347 41.89382)
2,Devon Snyder,2550,1259,Milton,SurfaceProblem,41.893982,-88.103340,xzribizdMX2eFxY6W8c7Tg,None,2.0,f,"cracks,grass",None,POINT (-88.10334 41.89398)
3,Devon Snyder,2551,1259,Milton,SurfaceProblem,41.894131,-88.103302,JS7TZGSTE9PJI_DpazVx2g,None,1.0,f,grass,None,POINT (-88.10330 41.89413)
4,Devon Snyder,2552,1259,Milton,SurfaceProblem,41.894184,-88.103271,JS7TZGSTE9PJI_DpazVx2g,None,2.0,f,"grass,bumpy",None,POINT (-88.10327 41.89418)


In [48]:
#count the cluster numbers for each label type
def count_clusters(labels, label_type_id):
    df = cluster_label_type_at_index(labels,label_type_id)[2]
    df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
    df1 = df[df['clustered'] == True]
    return df1.cluster_id.nunique()

In [49]:
cluster_count_chicago = {}
for i in [0,1,2,3,4,5,6,7,8,9]:
    clusters =  count_clusters(chicago_labels, i)
    cluster_count_chicago[label_types[i]] = clusters

In [50]:
cluster_count_chicago

{'CurbRamp': 333,
 'NoSidewalk': 67,
 'Problem': 131,
 'Occlusion': 1,
 'SurfaceProblem': 115,
 'Obstacle': 13,
 'Other': 0,
 'NoCurbRamp': 1,
 'Crosswalk': 91,
 'Signal': 103}

In [ ]:
#count the numbers for eanch label type
chicago_labels.label_type.value_counts()

In [ ]:
#read in chicago streets
chicago_streets = gpd.read_file("data/round-2/chicago-streets-230608.geojson")

In [ ]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to chicago_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [ ]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [ ]:
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    data_matrix = irr.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')

    return alpha.round(3)
    

In [ ]:
alpha_chicago = {}
for i in [0,1,2,4,5,8,9]:
    alpha = krippendorff_alpha(chicago_labels,i, chicago_streets, 3435)
    alpha_chicago[label_types[i]] = alpha

In [39]:
#30 ft segments
alpha_chicago

{'CurbRamp': 0.934,
 'NoSidewalk': 0.734,
 'Problem': 0.598,
 'SurfaceProblem': 0.584,
 'Obstacle': 0.595,
 'Crosswalk': 0.922,
 'Signal': 0.99}

surface problem and obstacle for severity level > = 4

In [40]:
#take out label type = surfaceproblem and obstacle
chicago_problems= chicago_labels[(chicago_labels['label_type'] == 'SurfaceProblem') | (chicago_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
chicago_problems = chicago_problems[chicago_problems['severity'] >= 4]
chicago_problems

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
29,Devon Snyder,2579,870,Milton,Obstacle,41.889069,-88.106689,G31WYuAr_xrgi2elpukCwg,None,5.0,t,construction,None,POINT (-88.10669 41.88907)
110,Devon Snyder,2687,940,Naperville,SurfaceProblem,41.801205,-88.153229,Yc7bAyE1oamSSfDM161CYQ,None,4.0,f,"cracks,very broken",None,POINT (-88.15323 41.80120)
233,mikey,2816,1256,Milton,Obstacle,41.889652,-88.106651,W1BXBBCz_AVeEWznKkVPow,None,5.0,f,construction,None,POINT (-88.10665 41.88965)
293,mikey,2877,1014,Milton,Obstacle,41.890877,-88.106155,Ed_OwJO-7XOpuh3iPbLX-w,None,5.0,f,construction,None,POINT (-88.10616 41.89088)
336,mikey,2924,940,Naperville,SurfaceProblem,41.801205,-88.153229,Yc7bAyE1oamSSfDM161CYQ,None,4.0,f,cracks,None,POINT (-88.15323 41.80120)
341,mikey,2930,934,Naperville,SurfaceProblem,41.801243,-88.150719,5z8njsTLwSvNnmnsruxHOA,None,4.0,f,uneven/slanted,None,POINT (-88.15072 41.80124)
556,Devon Snyder,3153,867,Downers Grove North,Obstacle,41.773682,-87.980186,VqXsMyejuv9LSqxptxTVgA,None,5.0,t,vegetation,None,POINT (-87.98019 41.77368)
835,Devon Snyder,3436,1010,Downers Grove North,SurfaceProblem,41.762501,-87.940704,xm8ep910NYSlFJj7z5JKJA,None,4.0,f,height difference,None,POINT (-87.94070 41.76250)
836,Devon Snyder,3437,1010,Downers Grove North,SurfaceProblem,41.762482,-87.940773,xm8ep910NYSlFJj7z5JKJA,None,4.0,f,height difference,None,POINT (-87.94077 41.76248)
888,Devon Snyder,3492,989,Downers Grove North,Obstacle,41.760040,-87.935394,IpAWF-Vq9R_FhPocXbUQHw,None,4.0,t,construction,None,POINT (-87.93539 41.76004)


In [ ]:
label_types

In [41]:
alpha_chicago_problems = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(chicago_problems,i, chicago_streets, 3435)
    alpha_chicago_problems[label_types[i]] = alpha

In [42]:
alpha_chicago_problems

{'Problem': 0.376, 'SurfaceProblem': 0.197, 'Obstacle': 0.544}

In [ ]:
# #plot severity counts for label type = surfaceproblem and obstacle
# sns.countplot(x='severity', hue='label_type', data=chicago_problems)
# #add title
# plt.title('Severity Counts for Surface Problem and Obstacle Labels Chicago')


In [ ]:
for i in [0,1,2,4,5,7,8,9]:
    labels= map_clusters(chicago_labels,i)
    map = chicago_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/chicago_" + label_types[i]+ "_230608.html")



In [ ]:
for i in [2,4,5]:
    labels= map_clusters(chicago_problems,i)
    map = chicago_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/chicago_wo_sev1_"+label_types[i]+"_230608.html")